# Batch Process Source HEC-RAS Models
Generate FIM Libraries for multiple HEC-RAS Models

In [ ]:
%load_ext autoreload
%autoreload 2

from scripts.processing import *

### `Parameters`

In [ ]:
# stac_endpoint = "https://stac2.dewberryanalytics.com"  # STAC API endpoint
# stac_collection = "12030106_EastForkTrinity"  # Collection name

source_models_dir = r"D:\Users\abdul.siddiqui\workbench\projects\test_local\source_models"
submodels_dir = r"D:\Users\abdul.siddiqui\workbench\projects\test_local\submodels"
library_dir = r"D:\Users\abdul.siddiqui\workbench\projects\test_local\library"

db_path = r"D:\Users\abdul.siddiqui\workbench\projects\test_local\library.db"
nwm_flowlines_path = r"D:\Users\abdul.siddiqui\workbench\projects\ripple_1d_outputs\nwm_flowlines.parquet"

### Get Working Models

In [ ]:
# models = get_models(stac_endpoint, stac_collection)
# or get based on some geographic unit
models = ["Baxter"]

### Load Conflation Information to Database

In [ ]:
load_conflation(models, source_models_dir, db_path)

### Update `network_to_id` Table in Database
Build a modified network by skipping over reaches eclipsed by adjacent reaches

In [ ]:
%%time
update_network(db_path)

### Get Working Reaches

In [ ]:
reaches = get_reaches_by_models(db_path, models)

### Run `extract_submodel`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(reaches, "extract_submodel", db_path, source_models_dir, submodels_dir)
if len(failed_reaches) + len(not_accepted_reaches) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `create_ras_terrain`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(reaches, "create_ras_terrain", db_path, source_models_dir, submodels_dir)
if len(failed_reaches) + len(not_accepted_reaches) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `create_model_run_normal_depth`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(reaches, "create_model_run_normal_depth", db_path, source_models_dir, submodels_dir)
if len(failed_reaches) + len(not_accepted_reaches) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `run_incremental_normal_depth`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(reaches, "run_incremental_normal_depth", db_path, source_models_dir, submodels_dir)
if len(failed_reaches) + len(not_accepted_reaches) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `run_known_wse` for Network

In [ ]:
execute_kwse_for_network([(reach[0], None) for reach in reaches if reach[1] is None], submodels_dir, db_path, False)

## Merge Libraries and Rating Curves

In [ ]:
move_fims_to_library(submodels_dir, library_dir)

In [ ]:
load_all_rating_curves(library_dir, db_path)